# A1111 Model Installer — SD 1.5 (UI)

Installs **SD 1.5** assets for Automatic1111 using a simple UI:

- **Checkpoints** (paste URLs) → `/workspace/a1111/models/Stable-diffusion/`
- **VAE (optional)** → `/workspace/a1111/models/VAE/`
- **ControlNet (SD15)** → `/workspace/a1111/models/ControlNet/`

**Tokens (optional, not required):**
- `HF_TOKEN` — for gated Hugging Face models.
- `CIVITAI_TOKEN` — for private/NSFW or rate-limited Civitai models.

Re-running is safe: existing files are skipped; downloads resume when possible.

In [ ]:
from pathlib import Path
import os

DATA_ROOT = Path(os.environ.get("WEBUI_ROOT", "/workspace/a1111"))
CKPT_DIR = DATA_ROOT / "models/Stable-diffusion"
VAE_DIR  = DATA_ROOT / "models/VAE"
CN_DIR   = DATA_ROOT / "models/ControlNet"

for d in (CKPT_DIR, VAE_DIR, CN_DIR):
    d.mkdir(parents=True, exist_ok=True)

print("Data root:", DATA_ROOT)
print("↳ Checkpoints:", CKPT_DIR)
print("↳ VAE:", VAE_DIR)
print("↳ ControlNet (SD15):", CN_DIR)

## Helpers (progress + optional tokens)
Supports Hugging Face (repo + filename) and direct URLs (e.g. Civitai). Uses tokens if present.

In [ ]:
import os, shutil, requests, time\nfrom tqdm import tqdm\nfrom huggingface_hub import hf_hub_download\nfrom pathlib import Path\n\n# Optional tokens (safe to be missing)\nHF_TOKEN = os.environ.get("HF_TOKEN")\nCIVITAI_TOKEN = os.environ.get("CIVITAI_TOKEN")\n\ndef human(n: int) -> str:\n    for unit in ["B","KB","MB","GB"]:\n        if n < 1024: return f"{n:.1f} {unit}"\n        n /= 1024\n    return f"{n:.1f} TB"\n\ndef download_direct(url: str, dest: Path) -> Path:\n    dest.parent.mkdir(parents=True, exist_ok=True)\n    if dest.exists():\n        print("✔ Exists:", dest); return dest\n    headers = {"Authorization": f"Bearer {CIVITAI_TOKEN}"} if CIVITAI_TOKEN else {}\n    try:\n        r = requests.get(url, headers=headers, stream=True, timeout=60, allow_redirects=True)\n        r.raise_for_status()\n    except requests.HTTPError as e:\n        print(f"⚠️  Direct download failed: {e}")\n        print("   If this is a private/NSFW Civitai model, set CIVITAI_TOKEN in RunPod env.")\n        return dest\n    total = int(r.headers.get('Content-Length', 0))\n    tmp = dest.with_suffix(dest.suffix + ".part")\n    with open(tmp, 'wb') as f, tqdm(total=total, unit='B', unit_scale=True, desc=dest.name, ncols=80) as bar:\n        for chunk in r.iter_content(chunk_size=1024*1024):\n            if chunk:\n                f.write(chunk); bar.update(len(chunk))\n    tmp.replace(dest)\n    try:\n        print("✔ Saved:", dest, f"({human(dest.stat().st_size)})")\n    except Exception:\n        print("✔ Saved:", dest)\n    return dest\n\ndef download_hf(repo_id: str, filename: str, dest_dir: Path, dest_name: str|None=None):\n    dest = dest_dir / (dest_name or filename)\n    dest.parent.mkdir(parents=True, exist_ok=True)\n    if dest.exists():\n        print("✔ Exists:", dest); return dest\n    print(f"→ HF: {repo_id}/{filename}")\n    try:\n        path = hf_hub_download(\n            repo_id=repo_id,\n            filename=filename,\n            token=HF_TOKEN,\n            local_dir=dest_dir,\n            local_dir_use_symlinks=False,\n            resume_download=True,\n        )\n        if Path(path) != dest:\n            shutil.copy2(path, dest)\n        print("✔ HF downloaded:", dest, f"({human(dest.stat().st_size)})")\n    except Exception as e:\n        print(f"⚠️  Hugging Face download failed: {e}")\n        print("   If this is a gated model, set HF_TOKEN in RunPod env and accept the license on HF.")\n    return dest\n\nprint("Helpers loaded. HF_TOKEN=", bool(HF_TOKEN), "CIVITAI_TOKEN=", bool(CIVITAI_TOKEN))\n

## Select and download assets
- Paste **SD 1.5 checkpoint** URLs (one per line). 
- Optional VAE and ControlNet choices below.

In [ ]:
import ipywidgets as W
from IPython.display import display, Markdown

vae_checkbox = W.Checkbox(description='Install SD 1.5 MSE VAE (stabilityai/sd-vae-ft-mse)', value=False)

cn_options = [("Canny", "lllyasviel/control_v11p_sd15_canny"),
                        ("Depth", "lllyasviel/control_v11p_sd15_depth"),
                        ("SoftEdge", "lllyasviel/control_v11p_sd15_softedge"),
                        ("LineArt", "lllyasviel/control_v11p_sd15_lineart"),
                        ("NormalBae", "lllyasviel/control_v11p_sd15_normalbae"),
                        ("OpenPose", "lllyasviel/control_v11p_sd15_openpose")]

prechecked = {"Canny","SoftEdge","OpenPose"}
cn_checkboxes = [W.Checkbox(description=label, value=(label in prechecked)) for label,_ in cn_options]

ckpt_text = W.Textarea(
  value='',
  placeholder='Paste one or more SD 1.5 .safetensors URLs here (one per line).',
  layout=W.Layout(width='100%', height='140px')
)

btn = W.Button(description='Download Selected', button_style='success', icon='download')
out = W.Output()

def on_click(_):
  out.clear_output()
  with out:
    errors = []
    if vae_checkbox.value:
      try:
        download_hf('stabilityai/sd-vae-ft-mse', 'vae-ft-mse-840000-ema-pruned.safetensors', VAE_DIR)
      except Exception as e: errors.append(('VAE', str(e)))

    for cb,(label,repo) in zip(cn_checkboxes, cn_options):
      if not cb.value: continue
      try:
        download_hf(repo, 'diffusion_pytorch_model.safetensors', CN_DIR, f'control_v11p_sd15_{label.lower()}.safetensors')
      except Exception as e: errors.append((f'ControlNet {label}', str(e)))

    for url in [u.strip() for u in ckpt_text.value.splitlines() if u.strip()]:
      try:
        download_direct(url, CKPT_DIR)
      except Exception as e: errors.append((url, str(e)))

    print('\n=== Summary ===')
    for p in sorted(list(CKPT_DIR.glob('*'))+list(VAE_DIR.glob('*'))+list(CN_DIR.glob('*'))):
      try: print(f'✔ {p.name:45s} {os.path.getsize(p):,} B')
      except Exception: print('✔', p.name)
    if errors:
      print('\nErrors:')
      for name,msg in errors: print('✖', name, '→', msg)

btn.on_click(on_click)
display(Markdown('### SD 1.5 Downloads'))
display(Markdown('**Checkpoints (paste URLs):**'))
display(ckpt_text)
display(Markdown('**VAE (optional):**'))
display(vae_checkbox)
display(Markdown('**ControlNet (SD15):**'))
for cb in cn_checkboxes: display(cb)
display(btn,out)

## (Optional) Restart the WebUI

In [ ]:
import subprocess, shlex, time
print('Attempting to restart the A1111 WebUI service via supervisord …')
try:
  cmd='supervisorctl -c /etc/supervisord.conf restart a1111'
  out=subprocess.run(shlex.split(cmd),capture_output=True,text=True,timeout=30)
  if out.returncode==0 and 'a1111:' in (out.stdout or ''):
    print((out.stdout or out.stderr).strip() or '✔ Restarted via supervisorctl')
  else:
    print('Supervisorctl unavailable; forcing process restart…')
    subprocess.run(['pkill','-f','launch.py'],check=False)
    time.sleep(5)
    print('✔ WebUI killed; supervisord will auto-restart. Refresh in ~10–15 s.')
except Exception as e:
  print('⚠️ Restart error:',e)
  print('Manual command:')
  print('supervisorctl -c /etc/supervisord.conf restart a1111')